In [1]:
import os
import gc

# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 결측치 시각화를 위한 라이브러리
import missingno

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

In [2]:
# parquet 파일 데이터를 읽어온다.
df = pd.read_parquet('open/train/3.승인매출정보/201807_train_승인매출정보.parquet')
df

,기준년월,ID,최종이용일자_기본,최종이용일자_신판,최종이용일자_CA,최종이용일자_카드론,최종이용일자_체크,최종이용일자_일시불,최종이용일자_할부,이용건수_신용_B0M,...,승인거절건수_한도초과_B0M,승인거절건수_BL_B0M,승인거절건수_입력오류_B0M,승인거절건수_기타_B0M,승인거절건수_R3M,승인거절건수_한도초과_R3M,승인거절건수_BL_R3M,승인거절건수_입력오류_R3M,승인거절건수_기타_R3M,이용금액대
0,201807,TRAIN_000000,20180719,20180713,20180719,10101,20180203,20180709,20180713,11,...,0,0,0,0,3,3,0,0,0,01.100만원+
1,201807,TRAIN_000001,20180719,20180719,20170728,20170327,10101,20180719,20171231,13,...,0,0,0,0,3,3,0,0,0,03.30만원+
2,201807,TRAIN_000002,20180706,20180706,20180706,20151119,20141230,20180706,20180627,12,...,0,0,0,0,0,0,0,0,0,01.100만원+
3,201807,TRAIN_000003,20180721,20180715,20180721,10101,20141111,20180704,20180715,6,...,0,0,0,0,3,3,0,0,0,01.100만원+
4,201807,TRAIN_000004,20180124,20180124,10101,10101,20180512,20180124,10101,-2,...,0,0,0,0,0,0,0,0,0,09.미사용
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,201807,TRAIN_399995,20170217,20170217,10101,10101,20180719,20170217,20160501,0,...,0,0,0,0,0,0,0,0,0,09.미사용
399996,201807,TRAIN_399996,20180731,20180731,10101,20170112,10101,20180731,20180112,21,...,0,0,0,0,0,0,0,0,0,01.100만원+
399997,201807,TRAIN_399997,20180726,20180726,10101,10101,20131124,20180720,20180726,16,...,0,0,0,0,0,0,0,0,0,02.50만원+
399998,201807,TRAIN_399998,20161224,20161224,10101,10101,10101,20161224,20150122,-1,...,0,0,0,0,0,0,0,0,0,09.미사용


In [3]:
# 파일이 있는 폴더 경로
folder_path = 'open/train/3.승인매출정보'

# 파일 목록
file_names = [
    "201807_train_승인매출정보.parquet",
    "201808_train_승인매출정보.parquet",
    "201809_train_승인매출정보.parquet",
    "201810_train_승인매출정보.parquet",
    "201811_train_승인매출정보.parquet",
    "201812_train_승인매출정보.parquet"
]

# 파일을 읽어서 하나의 DataFrame으로 합치기
dfs = [pd.read_parquet(os.path.join(folder_path, file)) for file in file_names]
df1 = pd.concat(dfs, ignore_index=True)

df1

,기준년월,ID,최종이용일자_기본,최종이용일자_신판,최종이용일자_CA,최종이용일자_카드론,최종이용일자_체크,최종이용일자_일시불,최종이용일자_할부,이용건수_신용_B0M,...,승인거절건수_한도초과_B0M,승인거절건수_BL_B0M,승인거절건수_입력오류_B0M,승인거절건수_기타_B0M,승인거절건수_R3M,승인거절건수_한도초과_R3M,승인거절건수_BL_R3M,승인거절건수_입력오류_R3M,승인거절건수_기타_R3M,이용금액대
0,201807,TRAIN_000000,20180719,20180713,20180719,10101,20180203,20180709,20180713,11,...,0,0,0,0,3,3,0,0,0,01.100만원+
1,201807,TRAIN_000001,20180719,20180719,20170728,20170327,10101,20180719,20171231,13,...,0,0,0,0,3,3,0,0,0,03.30만원+
2,201807,TRAIN_000002,20180706,20180706,20180706,20151119,20141230,20180706,20180627,12,...,0,0,0,0,0,0,0,0,0,01.100만원+
3,201807,TRAIN_000003,20180721,20180715,20180721,10101,20141111,20180704,20180715,6,...,0,0,0,0,3,3,0,0,0,01.100만원+
4,201807,TRAIN_000004,20180124,20180124,10101,10101,20180512,20180124,10101,-2,...,0,0,0,0,0,0,0,0,0,09.미사용
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,20181220,20181220,10101,10101,20181212,20181220,20160501,2,...,0,0,0,0,0,0,0,0,0,09.미사용
2399996,201812,TRAIN_399996,20181202,20181202,10101,20170112,10101,20181202,20180112,10,...,0,0,0,0,0,0,0,0,0,01.100만원+
2399997,201812,TRAIN_399997,20181230,20181230,10101,10101,20131124,20181230,20180919,10,...,0,0,0,0,0,0,0,0,0,02.50만원+
2399998,201812,TRAIN_399998,20161224,20161224,10101,10101,10101,20161224,20150122,-2,...,0,0,0,0,0,0,0,0,0,09.미사용


### EDA

In [4]:
# 데이터 프레임 전체 정보 확인
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Columns: 406 entries, 기준년월 to 이용금액대
dtypes: float64(3), int64(385), object(18)
memory usage: 7.3+ GB


In [5]:
# 데이터 타입(형식) 확인
print(df1.dtypes)

기준년월                int64
ID                 object
최종이용일자_기본           int64
최종이용일자_신판           int64
최종이용일자_CA           int64
                    ...  
승인거절건수_한도초과_R3M     int64
승인거절건수_BL_R3M       int64
승인거절건수_입력오류_R3M     int64
승인거절건수_기타_R3M       int64
이용금액대              object
Length: 406, dtype: object


In [6]:
# 수치형 변수만 요약(변수의 최대/최소값)
df.describe()

,기준년월,최종이용일자_기본,최종이용일자_신판,최종이용일자_CA,최종이용일자_카드론,최종이용일자_체크,최종이용일자_일시불,최종이용일자_할부,이용건수_신용_B0M,이용건수_신판_B0M,...,승인거절건수_B0M,승인거절건수_한도초과_B0M,승인거절건수_BL_B0M,승인거절건수_입력오류_B0M,승인거절건수_기타_B0M,승인거절건수_R3M,승인거절건수_한도초과_R3M,승인거절건수_BL_R3M,승인거절건수_입력오류_R3M,승인거절건수_기타_R3M
count,400000.0,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,400000.000000,400000.000000,...,400000.000000,400000.000000,400000.000000,400000.0,400000.0,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000
mean,201807.0,1.938216e+07,1.937439e+07,6.435386e+06,3.466644e+06,7.408213e+06,1.932841e+07,1.454768e+07,15.773220,15.690905,...,0.006868,0.006650,0.000218,0.0,0.0,0.359208,0.199648,0.046790,0.000015,0.112755
std,0.0,3.928628e+06,3.946677e+06,9.389315e+06,7.596589e+06,9.714134e+06,4.052097e+06,9.040274e+06,20.176325,20.162638,...,0.118175,0.115134,0.025543,0.0,0.0,1.029473,0.746903,0.425983,0.003873,0.458783
min,201807.0,1.010100e+04,1.010100e+04,1.010100e+04,1.010100e+04,1.010100e+04,1.010100e+04,1.010100e+04,-3.000000,-3.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,201807.0,2.018070e+07,2.018070e+07,1.010100e+04,1.010100e+04,1.010100e+04,2.018070e+07,1.010100e+04,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,201807.0,2.018072e+07,2.018072e+07,1.010100e+04,1.010100e+04,1.010100e+04,2.018072e+07,2.016071e+07,8.000000,7.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,201807.0,2.018073e+07,2.018073e+07,2.014043e+07,1.010100e+04,2.015031e+07,2.018073e+07,2.018051e+07,25.000000,24.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
max,201807.0,2.018073e+07,2.018073e+07,2.018073e+07,2.018073e+07,2.018073e+07,2.018073e+07,2.018073e+07,90.000000,89.000000,...,5.000000,2.000000,3.000000,0.0,0.0,10.000000,3.000000,4.000000,1.000000,2.000000


In [7]:
# 전체 결측치 개수
df1.isnull().sum().sum()

28929002

In [8]:
# 각 열의 결측치 개수
df1.isnull().sum()

기준년월               0
ID                 0
최종이용일자_기본          0
최종이용일자_신판          0
최종이용일자_CA          0
                  ..
승인거절건수_한도초과_R3M    0
승인거절건수_BL_R3M      0
승인거절건수_입력오류_R3M    0
승인거절건수_기타_R3M      0
이용금액대              0
Length: 406, dtype: int64

In [9]:
# 결측치가 있는 열 표시
null_cols = df1.columns[df1.isnull().any()]
null_cols

Index(['_1순위업종', '_2순위업종', '_3순위업종', '_1순위쇼핑업종', '_2순위쇼핑업종', '_3순위쇼핑업종',
       '_1순위교통업종', '_2순위교통업종', '_3순위교통업종', '_1순위여유업종', '_2순위여유업종', '_3순위여유업종',
       '_1순위납부업종', '_2순위납부업종', '_3순위납부업종', '최종카드론_금융상환방식코드', '최종카드론_신청경로코드',
       '최종카드론_대출일자'],
      dtype='object')

In [10]:
#결측치가 있는 열만 필터링
df1[null_cols].isnull().sum()

_1순위업종             539992
_2순위업종             912725
_3순위업종            1107898
_1순위쇼핑업종           922663
_2순위쇼핑업종          1135042
_3순위쇼핑업종          1312267
_1순위교통업종          1164494
_2순위교통업종          1656423
_3순위교통업종          2045455
_1순위여유업종          1987260
_2순위여유업종          2302286
_3순위여유업종          2377725
_1순위납부업종          1216263
_2순위납부업종          2033640
_3순위납부업종          2310187
최종카드론_금융상환방식코드    1958126
최종카드론_신청경로코드      1958226
최종카드론_대출일자        1988330
dtype: int64

In [11]:
df1[null_cols].dtypes

_1순위업종             object
_2순위업종             object
_3순위업종             object
_1순위쇼핑업종           object
_2순위쇼핑업종           object
_3순위쇼핑업종           object
_1순위교통업종           object
_2순위교통업종           object
_3순위교통업종           object
_1순위여유업종           object
_2순위여유업종           object
_3순위여유업종           object
_1순위납부업종           object
_2순위납부업종           object
_3순위납부업종           object
최종카드론_금융상환방식코드    float64
최종카드론_신청경로코드       object
최종카드론_대출일자        float64
dtype: object

In [12]:
# 각 컬럼별 결측치 현황을 막대 그래프 형태로 확인해본다.
# 결측치를 제외한 데이터 개수
missingno.bar(df1)
plt.show()

KeyboardInterrupt: 

In [13]:
df1[null_cols].head()

,_1순위업종,_2순위업종,_3순위업종,_1순위쇼핑업종,_2순위쇼핑업종,_3순위쇼핑업종,_1순위교통업종,_2순위교통업종,_3순위교통업종,_1순위여유업종,_2순위여유업종,_3순위여유업종,_1순위납부업종,_2순위납부업종,_3순위납부업종,최종카드론_금융상환방식코드,최종카드론_신청경로코드,최종카드론_대출일자
0,납부,쇼핑,사교활동,쇼핑기타,None,None,택시,버스지하철,None,None,None,None,보험료,납부기타,None,NaN,None,NaN
1,쇼핑,납부,교통,도소매,슈퍼마켓,편의점,주유,철도버스,None,None,None,None,통신비,None,None,5.0,8,20170327.0
2,쇼핑,사교활동,교통,온라인,도소매,마트,주유,None,None,None,None,None,None,None,None,5.0,8,20151119.0
3,쇼핑,납부,사교활동,마트,슈퍼마켓,None,택시,None,None,None,None,None,보험료,None,None,NaN,None,NaN
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN


In [14]:
# 결측치 제거
df2 = df1.drop(columns=null_cols)

In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Columns: 388 entries, 기준년월 to 이용금액대
dtypes: float64(1), int64(385), object(2)
memory usage: 6.9+ GB


In [20]:
df2.isna().sum().sum()

0